In [4]:
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
%matplotlib inline

In [5]:
image_size = 32
print([x[0] for x in os.walk("~")])
path = 'images/train/REAL'
img_names = os.listdir(path)

batch_size = 64


X_train = np.empty(batch_size, image_size, image_size, 3)
for idx, name in enumerate(img_names):
    img_name = path + name
    # Use you favourite library to load the image
    image = plt.imread(img_name)
    X_train[idx] = image
print(f"Shape of training data: {X_train.shape}")
print(f"Data type: {type(X_train)}")

[]


IsADirectoryError: [Errno 21] Is a directory: 'images/test/REAL'